In [1]:
from pandas import DataFrame, Series
import pandas as pd
import sys
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Data Aggregation

By aggregation, I am generally referring to any data transformation that produces scalar
values from arrays. In the examples above I have used several of them, such as mean,
count, min and sum. You may wonder what is going on when you invoke mean() on a
GroupBy object. Many common aggregations, such as those found in Table 9-1, have
optimized implementations that compute the statistics on the dataset in place. However,
you are not limited to only this set of methods. You can use aggregations of your own devising and additionally call any method that is also defined on the grouped
object. For example, as you recall quantile computes sample quantiles of a Series or a
DataFrame’s columns 1:

In [4]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})

In [5]:
df

,data1,data2,key1,key2
0,1.691915,-0.150375,a,one
1,0.035286,0.797915,a,two
2,2.645413,-1.249537,b,one
3,-0.645075,-0.926750,b,two
4,-2.411390,-1.640924,a,one


In [6]:
grouped = df.groupby('key1')

In [7]:
grouped['data1'].quantile(0.9)

key1
a    1.360589
b    2.316364
Name: data1, dtype: float64

While quantile is not explicitly implemented for GroupBy, it is a Series method and
thus available for use. Internally, GroupBy efficiently slices up the Series, calls
piece.quantile(0.9) for each piece, then assembles those results together into the result
object.

To use your own aggregation functions, pass any function that aggregates an array to
the aggregate or agg method:

In [9]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [10]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4.103306,2.438838
b,3.290488,0.322787


You’ll notice that some methods like describe also work, even though they are not
aggregations, strictly speaking:

In [11]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.228063  2.064290 -2.411390 -1.188052  0.035286  0.863600   
b      2.0  1.000169  2.326726 -0.645075  0.177547  1.000169  1.822791   

               data2                                                   \
           max count      mean       std       min      25%       50%   
key1                                                                    
a     1.691915   3.0 -0.331128  1.229425 -1.640924 -0.89565 -0.150375   
b     2.645413   2.0 -1.088143  0.228245 -1.249537 -1.16884 -1.088143   

                          
           75%       max  
key1                      
a     0.323770  0.797915  
b    -1.007447 -0.926750

I will explain in more detail what has happened here in the next major section on groupwise
operations and transformations.

Table 9-1. Optimized groupby methods

Function name Description

count Number of non-NA values in the group

sum Sum of non-NA values

mean Mean of non-NA values

median Arithmetic median of non-NA values

std, var Unbiased (n - 1 denominator) standard deviation and variance

min, max Minimum and maximum of non-NA values

prod Product of non-NA values

first, last First and last non-NA values



To illustrate some more advanced aggregation features, I’ll use a less trivial dataset, a
dataset on restaurant tipping. I obtained it from the R reshape2 package; it was originally
found in Bryant & Smith’s 1995 text on business statistics (and found in the book’s
GitHub repository). After loading it with read_csv, I add a tipping percentage column
tip_pct.

In [17]:

tips = pd.read_csv('tips.csv')

In [18]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [19]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


## Column-wise and Multiple Function Application
As you’ve seen above, aggregating a Series or all of the columns of a DataFrame is a
matter of using aggregate with the desired function or calling a method like mean or
std. However, you may want to aggregate using a different function depending on the
column or multiple functions at once. Fortunately, this is straightforward to do, which
I’ll illustrate through a number of examples. First, I’ll group the tips by sex and smoker:

In [20]:
grouped = tips.groupby(['sex', 'smoker'])

In [21]:
grouped_pct = grouped['tip_pct']

In [22]:
grouped_pct

In [23]:
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame with
column names taken from the functions:

In [24]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

You don’t need to accept the names that GroupBy gives to the columns; notably
lambda functions have the name '<lambda>' which make them hard to identify (you can
see for yourself by looking at a function’s __name__ attribute). As such, if you pass a list
of (name, function) tuples, the first element of each tuple will be used as the DataFrame
column names (you can think of a list of 2-tuples as an ordered mapping):

In [25]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

With a DataFrame, you have more options as you can specify a list of functions to apply
to all of the columns or different functions per column. To start, suppose we wanted
to compute the same three statistics for the tip_pct and total_bill columns:

In [26]:
functions = ['count', 'mean', 'max']

In [27]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [28]:
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

As you can see, the resulting DataFrame has hierarchical columns, the same as you
would get aggregating each column separately and using concat to glue the results
together using the column names as the keys argument:

In [29]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

As above, a list of tuples with custom names can be passed:

In [30]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [31]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

Now, suppose you wanted to apply potentially different functions to one or more of
the columns. The trick is to pass a dict to agg that contains a mapping of column names
to any of the function specifications listed so far:

In [32]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

size   tip
sex    smoker            
Female No       140   5.2
       Yes       74   6.5
Male   No       263   9.0
       Yes      150  10.0

A DataFrame will have hierarchical columns only if multiple functions are applied to
at least one column.

## Returning Aggregated Data in “unindexed” Form

In all of the examples up until now, the aggregated data comes back with an index,
potentially hierarchical, composed from the unique group key combinations observed.
Since this isn’t always desirable, you can disable this behavior in most cases by passing
as_index=False to groupby:

In [33]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


Of course, it’s always possible to obtain the result in this format by calling
reset_index on the result.

NOTE
Using groupby in this way is generally less flexible; results with hierarchical
columns, for example, are not currently implemented as the
form of the result would have to be somewhat arbitrary.

